In [39]:
import os
import base64
import requests
import datetime
import json

# ——— Configuration ———
API_KEY = os.environ.get("WANDB_API_KEY")  # W&B API 키 (환경변수 설정 권장)
ENTITY = "horangi"                        # 조회할 W&B 엔티티(조직)명
BASE_URL = "https://api.wandb.ai/audit_logs"  # 다중테넌트 클라우드용 엔드포인트 (Enterprise)  [oai_citation:1‡docs.wandb.ai](https://docs.wandb.ai/platform/hosting/monitoring-usage/audit-logging)

# 조회할 날짜 설정 (예: 최근 7일)
NUM_DAYS = 7
start_date = (datetime.date.today() - datetime.timedelta(days=NUM_DAYS - 1)).strftime("%Y-%m-%d")
params = {
    "startDate": start_date,
    "numDays": NUM_DAYS,
    # "anonymize": "true",  # PII(개인식별정보)를 제외하려면 사용 (Self-Managed 또는 Dedicated 인스턴스에서 지원)  [oai_citation:2‡docs.wandb.ai](https://docs.wandb.ai/platform/hosting/monitoring-usage/audit-logging)
}

# ——— Build Authentication Header ———
# 공식 문서에서는 Basic 인증 방식 예시 제시됨: “Authorization: Basic <base64(username:API-KEY)>”  [oai_citation:3‡docs.wandb.ai](https://docs.wandb.ai/platform/hosting/monitoring-usage/audit-logging)
# 여기서는 username 부분에 빈 문자열 또는 "api" 등을 사용 가능 (설정에 따라 다름)
username = ""
auth_string = f"{username}:{API_KEY}"
b64_encoded = base64.b64encode(auth_string.encode("utf-8")).decode("utf-8")
headers = {
    "Authorization": f"Basic {b64_encoded}",
    "Accept": "application/json",
}

# ——— Construct URL ———
url = f"{BASE_URL}?entity={ENTITY}"
# append params
param_str = "&".join(f"{k}={v}" for k, v in params.items())
if param_str:
    url = f"{url}&{param_str}"

# ——— Execute Request ———
response = requests.get(url, headers=headers, timeout=30)

if response.status_code == 200:
    # 응답은 newline-separated JSON 객체들로 구성될 수 있음  [oai_citation:4‡docs.wandb.ai](https://docs.wandb.ai/platform/hosting/monitoring-usage/audit-logging?utm_source=chatgpt.com)
    raw = response.text.strip().splitlines()
    logs = [json.loads(line) for line in raw]
    print(f"✅ Retrieved {len(logs)} audit log entries")
    # 예시 출력
    for log in logs[:10]:
        print(f"{log.get('timestamp')} | {log.get('actor_email')} | {log.get('action')} | {log.get('entity_name')} | {log.get('project_name')}")
else:
    print(f"❌ Failed to fetch audit logs: {response.status_code} – {response.text}")

✅ Retrieved 1 audit log entries
None | None | None | None | None


In [46]:
import requests
import base64

# Setup authentication
api_key = os.environ.get("WANDB_API_KEY")
auth = base64.b64encode(f"api:{api_key}".encode()).decode()

# Configure endpoint and parameters
url = "https://api.wandb.ai/audit_logs"  # or api.wandb.ai/audit_logs for SaaS
params = {
    "numDays": 7,
    "anonymize": "true"  # optional, excludes PII
}

headers = {
    "Authorization": f"Basic {auth}"
}

response = requests.get(url, params=params, headers=headers)
print(response.text)

{"audit_logs":[],"range_start_time":"2025-10-30T00:00:00Z","range_end_time":"2025-11-07T00:00:00Z"}
